In [1]:
import util
from backtesting import Backtest
import pandas_ta as ta
import numpy as np

lista = ['BTCUSDT', 'ETHUSDT', 'BCHUSDT', 'XRPUSDT', 'LTCUSDT', 'ETCUSDT', 'LINKUSDT', 'ADAUSDT', 'BNBUSDT', 'DOGEUSDT', 'DOTUSDT', 
         'SOLUSDT', 'AVAXUSDT', 'NEARUSDT', 'FILUSDT', 'MATICUSDT', 'OPUSDT', 'FETUSDT', 'AGIXUSDT', 'ARBUSDT',  
         'SLPUSDT', 'MEMEUSDT',  '1000SATSUSDT', 'PIXELUSDT']
lista=['BTCUSDT']

suma=0.0
trades = 0
timeframe='1h'
limit= 1000

print(f"timeframe: {timeframe} - limit: {limit}")
for symbol in lista:
    try:
        data=util.obtiene_historial(symbol=symbol,timeframe=timeframe,limit=limit)    
        data['ema100']=ta.ema(data.Close, length=100)
        data['ema200']=ta.ema(data.Close, length=200)
        porcentaje_perdida = 1
        ##    obligatorios   --->>>  trade, stop_loss, take_profit, porcentajeentrada
        data['trade'] = np.where(util.crossover_dataframe(data.ema100, data.ema200),1,np.where(util.crossover_dataframe(data.ema200, data.ema100),-1,0))
        data['stop_loss'] = np.where(data.trade==1,data.ema200,np.where(data.trade==-1,data.ema200,0))
        variacion_hasta_stop_loss = abs(((data.stop_loss/data.Close)-1)*100)
        data['take_profit'] = np.where(data.trade==1,data.Close+(data.Close-data.ema200)*3,np.where(data.trade==-1,data.Close-(data.ema200-data.Close)*3,0))           
        data['porcentajeentrada'] = np.where(((porcentaje_perdida/variacion_hasta_stop_loss))>=1,0.99,((porcentaje_perdida/variacion_hasta_stop_loss)))        ##
        ###
        bt = Backtest(data,util.backtesting_config,cash=1000000,exclusive_orders=True)
        output = bt.run()
        if len(lista)==1:
            bt.plot(filename="graficos/"+symbol)
        print(f"Symbol: {symbol} - Return [%]: {round(output['Return [%]'],2)} - # Trades: {output['# Trades']} - Win Rate [%]: {output['Win Rate [%]']}")
        suma=suma+output['Return [%]']
        trades=trades+output['# Trades']
    except Exception as e:
        print(f"Error en {symbol}: {e}")
print(f"Suma total: {suma} - Trades total: {trades}")

timeframe: 1h - limit: 1000
Symbol: BTCUSDT - Return [%]: -0.52 - # Trades: 4 - Win Rate [%]: 25.0
Suma total: -0.5237692224117927 - Trades total: 4


In [3]:
import util
from backtesting import Backtest
import pandas_ta as ta
import numpy as np

lista = ['BTCUSDT', 'ETHUSDT', 'BCHUSDT', 'XRPUSDT', 'LTCUSDT', 'ETCUSDT', 'LINKUSDT', 'ADAUSDT', 'BNBUSDT', 'DOGEUSDT', 'DOTUSDT', 
         'SOLUSDT', 'AVAXUSDT', 'NEARUSDT', 'FILUSDT', 'MATICUSDT', 'OPUSDT', 'FETUSDT', 'AGIXUSDT', 'ARBUSDT',  
         'SLPUSDT', 'MEMEUSDT',  '1000SATSUSDT', 'PIXELUSDT']
#lista=['BTCUSDT']

suma=0.0
trades = 0
timeframe='1h'
limit= 1000

print(f"timeframe: {timeframe} - limit: {limit}")
for symbol in lista:
    try:
        data=util.obtiene_historial(symbol=symbol,timeframe=timeframe,limit=limit)    
        data['sma200']=ta.sma(data.Close, length=200)
        porcentaje_perdida = 1
        ##    obligatorios   --->>>  trade, stop_loss, take_profit, porcentajeentrada
        data['trade'] = np.where((data.Close.shift(-1) < data.Open.shift(-1))
                                 & (data.Close.shift(-2) < data.Open.shift(-2))
                                 & (data.Close.shift(-3) < data.Open.shift(-3))
                                 & (data.Close > data.sma200)
                                 ,1,0)
        data['stop_loss'] = data.sma200
        variacion_hasta_stop_loss = abs(((data.stop_loss/data.Close)-1)*100)
        data['take_profit'] = np.where(data.trade==1,data.Close+(data.Close-data.sma200)*3,np.where(data.trade==-1,data.Close-(data.sma200-data.Close)*3,0))           
        data['porcentajeentrada'] = np.where(((porcentaje_perdida/variacion_hasta_stop_loss))>=1,0.99,((porcentaje_perdida/variacion_hasta_stop_loss)))        ##
        ###
        bt = Backtest(data,util.backtesting_config,cash=1000000,exclusive_orders=True)
        output = bt.run()
        if len(lista)==1:
            bt.plot(filename="graficos/"+symbol)
        print(f"Symbol: {symbol} - Return [%]: {round(output['Return [%]'],2)} - # Trades: {output['# Trades']} - Win Rate [%]: {output['Win Rate [%]']}")
        suma=suma+output['Return [%]']
        trades=trades+output['# Trades']
    except Exception as e:
        print(f"Error en {symbol}: {e}")
print(f"Suma total: {suma} - Trades total: {trades}")

timeframe: 1h - limit: 1000
Symbol: BTCUSDT - Return [%]: -6.75 - # Trades: 11 - Win Rate [%]: 0.0
Symbol: ETHUSDT - Return [%]: -8.19 - # Trades: 18 - Win Rate [%]: 0.0
Symbol: BCHUSDT - Return [%]: -6.36 - # Trades: 15 - Win Rate [%]: 13.333333333333334
Symbol: XRPUSDT - Return [%]: -2.37 - # Trades: 4 - Win Rate [%]: 25.0
Symbol: LTCUSDT - Return [%]: -4.93 - # Trades: 11 - Win Rate [%]: 9.090909090909092
Symbol: ETCUSDT - Return [%]: -10.3 - # Trades: 16 - Win Rate [%]: 6.25
Symbol: LINKUSDT - Return [%]: -9.63 - # Trades: 19 - Win Rate [%]: 5.263157894736842
Symbol: ADAUSDT - Return [%]: -11.63 - # Trades: 19 - Win Rate [%]: 5.263157894736842
Symbol: BNBUSDT - Return [%]: -2.69 - # Trades: 6 - Win Rate [%]: 16.666666666666664
Symbol: DOGEUSDT - Return [%]: -6.08 - # Trades: 9 - Win Rate [%]: 11.11111111111111
Symbol: DOTUSDT - Return [%]: -7.18 - # Trades: 12 - Win Rate [%]: 8.333333333333332
Symbol: SOLUSDT - Return [%]: -8.18 - # Trades: 13 - Win Rate [%]: 0.0
Symbol: AVAXUSDT -